In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.layers import TimeDistributed, Dense, Embedding, LSTM, Reshape, GRU
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
import shutil
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, LearningRateScheduler
from sklearn.model_selection import train_test_split
from distutils.dir_util import copy_tree
import json
import math
import errno
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [2]:
LEARNING_RATE = 0.0002
# Scheduling a learning-rate to produce different effects for gradient of loss wrt to the weights
# Different Learning Rates will affect the model differently by updating different % of weights of the model.
START_CHAR = "\t"
END_CHAR = "\n"
LSTM_UNITS = 300
DENSE_UNITS = 100
VOCAB_SIZE = 70
PROB_THRESHOLD = 1e-9
VERBOSITY = 1
EPOCHS = 50
MAX_LENGTH = 32
EMBEDDING_DIMENSION = 16
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 5000
EMBEDDING_METADATA = 'metadata.tsv'
PREFETCH_BATCHES = 10
METRICS = [tf.keras.metrics.SparseCategoricalCrossentropy(), 'accuracy']

In [3]:
# Preprocessing the inputs and the outputs for the model --- 
# For Example password 'passwd'
# Adding a \t at the Start for the input - \tpasswd 
# Adding a \n at the End for the model output - passwd\n
# Here \t predicts p, p predicts a and so on..
def loadPreprocessInputOutput(FILE_NAME, START_CHAR, END_CHAR):
    try:
        assert os.path.isfile(FILE_NAME)
        assert isinstance(START_CHAR, str)
        assert isinstance(END_CHAR, str)
    except:
        raise Exception("Incorrect Inputs. Try again.")
        return
    inputPasswords = []
    outputPasswords = []
    listPasswords = []
    with open(FILE_NAME, "r") as pass_file:
        while(True):
            single_pass = pass_file.readline().rstrip("\n")
            if(single_pass == ""):
                break
            else:
                inputPasswords.append(START_CHAR + single_pass)
                outputPasswords.append(single_pass + END_CHAR)
                listPasswords.append(START_CHAR + single_pass + END_CHAR)
        return(inputPasswords, outputPasswords, listPasswords)

In [11]:
inputPasswords, outPasswords, listPasswords = loadPreprocessInputOutput("..\\Embedding\\Data\\ascii_rockyou_less_than_thirty_two_cleaned.txt", START_CHAR, END_CHAR)

In [5]:
# Sanity Check
print(inputPasswords[5:10])
print(len(inputPasswords))
print(outPasswords[5:10])
print(len(outPasswords))
print(listPasswords[5:10])
print(len(listPasswords))

['\tprincess', '\t1234567', '\trockyou', '\t12345678', '\tabc123']
14105697
['princess\n', '1234567\n', 'rockyou\n', '12345678\n', 'abc123\n']
14105697
['\tprincess\n', '\t1234567\n', '\trockyou\n', '\t12345678\n', '\tabc123\n']
14105697


In [4]:
# Save the passwords:
# Do not open file - will not be shown properly
def writePreprocessedPasswordFromList(passwordList, typeList):
    passwordFileName = typeList + "_preprocessed.txt"
    with open(passwordFileName, "w") as pass_file:
        for password in passwordList:
            if(typeList.lower() == "input"):
                pass_file.write(password+"\n")
            else:
                pass_file.write(password)
    print(f"{typeList} passwords written completely to : {passwordFileName}")

In [ ]:
# Write the passwords
writePreprocessedPasswordFromList(inputPasswords, "input")
writePreprocessedPasswordFromList(outPasswords, "output")
writePreprocessedPasswordFromList(listPasswords, "list")

In [5]:
# Sanity Check
# Output will not be proper.
!head -n 5 input_preprocessed.txt
!head -n 5 output_preprocessed.txt
!head -n 5 list_preprocessed.txt
!wc -l input_preprocessed.txt
!wc -l output_preprocessed.txt
!wc -l list_preprocessed.txt

	123456
	12345
	123456789
	password
	iloveyou
123456
12345
123456789
password
iloveyou
	123456
	12345
	123456789
	password
	iloveyou
14105697 input_preprocessed.txt
14105697 output_preprocessed.txt
14105697 list_preprocessed.txt


In [5]:
# Specifying the Tokenizer -- leaving num_words as blank to include as many 
# unique characters as possible.
# Fit the tokenizer on the text and then save the tokenizer.
passwordTokenizer = Tokenizer(filters = "", lower = True, char_level = True)

In [6]:
# Utility Function to Save the Tokenizer Configuration
def saveTokenizer(TOKENIZER, OUTPUT_PATH):
    tokenizerConfigString = TOKENIZER.to_json()
    with open(OUTPUT_PATH+".json", "w") as op_file:
        op_file.write(tokenizerConfigString)

In [7]:
# Utility Function to Load the Tokenizer Configuration
def loadTokenizer(TOKENIZER_FILE_PATH):
    _, file_extension = os.path.splitext(TOKENIZER_FILE_PATH)
    if(file_extension != ".json"):
        raise Exception("Incorrect File.")
        return
    else:
        with open(TOKENIZER_FILE_PATH, "r") as tokenizer_cfg_file:
            tokenizer_config = tokenizer_cfg_file.read()
            tokenizer_cfg = json.loads(tokenizer_config)
            passwordTokenizer = tf.keras.preprocessing.text.tokenizer_from_json(json.dumps(tokenizer_cfg))
            return passwordTokenizer

In [15]:
# Fitting the Tokenizer on data:
passwordTokenizer.fit_on_texts(listPasswords)
passwordTokenizer.get_config()

{'num_words': None,
 'filters': '',
 'lower': True,
 'split': ' ',
 'char_level': True,
 'oov_token': None,
 'document_count': 14105697,
 'word_counts': '{"\\t": 14105697, "1": 6729506, "2": 5234401, "3": 3765169, "4": 3389487, "5": 3352338, "6": 3116090, "\\n": 14105697, "7": 3098762, "8": 3565308, "9": 3853241, "p": 1619704, "a": 8828625, "s": 4154066, "w": 799568, "o": 5173138, "r": 4576620, "d": 2484237, "i": 5553157, "l": 4460473, "v": 1050793, "e": 7203479, "y": 2373398, "u": 2307207, "n": 4827931, "c": 2608350, "k": 2012166, "b": 2110821, "g": 1717216, "m": 3205286, "j": 1237461, "h": 2335137, "q": 178503, "t": 3425154, "0": 5735322, "f": 981496, "z": 763424, "x": 479316, "!": 142923, ";": 12258, "-": 133010, "*": 123842, ".": 248717, "?": 18301, ",": 29750, "/": 48190, "#": 48873, "@": 107880, "$": 36029, "%": 10254, "^": 6394, "&": 26359, "+": 26989, "\'": 15335, "[": 7682, "]": 10802, "<": 9561, "_": 193006, ">": 2458, "=": 18365, "\\\\": 25832, "\\"": 3637, ":": 6858, "(": 1

In [16]:
# Save Tokenizer -- for later use and load it to avoid re-fitting
saveTokenizer(passwordTokenizer, "prototypeTokenizer")

In [8]:
passwordTokenizer = loadTokenizer("prototypeTokenizer.json")
# Sanity Check
passwordTokenizer.get_config()

{'num_words': None,
 'filters': '',
 'lower': True,
 'split': ' ',
 'char_level': True,
 'oov_token': None,
 'document_count': 14105697,
 'word_counts': '{"\\t": 14105697, "1": 6729506, "2": 5234401, "3": 3765169, "4": 3389487, "5": 3352338, "6": 3116090, "\\n": 14105697, "7": 3098762, "8": 3565308, "9": 3853241, "p": 1619704, "a": 8828625, "s": 4154066, "w": 799568, "o": 5173138, "r": 4576620, "d": 2484237, "i": 5553157, "l": 4460473, "v": 1050793, "e": 7203479, "y": 2373398, "u": 2307207, "n": 4827931, "c": 2608350, "k": 2012166, "b": 2110821, "g": 1717216, "m": 3205286, "j": 1237461, "h": 2335137, "q": 178503, "t": 3425154, "0": 5735322, "f": 981496, "z": 763424, "x": 479316, "!": 142923, ";": 12258, "-": 133010, "*": 123842, ".": 248717, "?": 18301, ",": 29750, "/": 48190, "#": 48873, "@": 107880, "$": 36029, "%": 10254, "^": 6394, "&": 26359, "+": 26989, "\'": 15335, "[": 7682, "]": 10802, "<": 9561, "_": 193006, ">": 2458, "=": 18365, "\\\\": 25832, "\\"": 3637, ":": 6858, "(": 1

In [32]:
def getLength(FILE_PATH, break_num = 0):
    count = 0
    with open(FILE_PATH, "r") as f:
        for count, _ in enumerate(f):
            pass
    count = count + 1
    if(break_num == 0):
        return count
    else:
        return break_num

In [9]:
# Function to preprocess and get validation data
def getValidationData(VALID_X_FILE_PATH, VALID_Y_FILE_PATH, VOCAB_SIZE, TOKENIZER, MAX_LENGTH, VALIDATION_BATCH_SIZE, TRAIN_ALL = None):
    if(TRAIN_ALL is not None):
        total_passwords = getLength(VALID_X_FILE_PATH, TRAIN_ALL)
    else:
        total_passwords = getLength(VALID_X_FILE_PATH)
    total_batches = math.floor(total_passwords / VALIDATION_BATCH_SIZE)
    total_passwords_to_read = total_batches * VALIDATION_BATCH_SIZE
    valid_passwords = []
    count = 0
    valid_y = []
    flag = True
    with open(VALID_X_FILE_PATH, "r") as valid_file:
        for _, password in enumerate(valid_file):
            if(_ < total_passwords_to_read):
                valid_passwords.append(password.rstrip("\n"))
    with open(VALID_Y_FILE_PATH, "r") as valid_true_file:
        for _, true_password in enumerate(valid_true_file):
            if(_ < total_passwords_to_read):
                valid_y.append(true_password)
    valid_encoded_passwords = TOKENIZER.texts_to_sequences(valid_passwords)
    valid_y_encoded_passwords = TOKENIZER.texts_to_sequences(valid_y)
    valid_padded_passwords = pad_sequences(valid_encoded_passwords, padding = "post", maxlen = (MAX_LENGTH + 1))
    valid_y_padded_passwords = pad_sequences(valid_y_encoded_passwords, padding = "post", maxlen = (MAX_LENGTH + 1))
    print(len(valid_padded_passwords))
    print(len(valid_y_padded_passwords))
    for x_valid_password, y_valid_password in zip(valid_padded_passwords, valid_y_padded_passwords):
        count += 1
        if(count % 500 == 0):
            # Give progress feedback
            print(f"Total passwords processed {count}")
        if(flag):
            final_x_valid = np.array(x_valid_password).reshape(1, (MAX_LENGTH + 1))
            temp_y_valid = np.array(y_valid_password).reshape(1, (MAX_LENGTH + 1))
            final_y_valid = to_categorical(temp_y_valid, num_classes = (VOCAB_SIZE + 1))
            flag = False
        else:
            final_x_valid = np.concatenate((final_x_valid, np.array(x_valid_password).reshape(1, (MAX_LENGTH + 1))), axis = 0)
            temp_y_valid = np.array(y_valid_password).reshape(1, (MAX_LENGTH + 1))                                 
            final_y_valid = np.concatenate((final_y_valid, to_categorical(temp_y_valid, num_classes = (VOCAB_SIZE + 1))), axis = 0)
    return(final_x_valid, final_y_valid) 

In [10]:
# Create an input pipeline for feeding encoded passwords -- 
# Pass train_passwords as None if you want to train on entire set of passwords mentioned in password_file
class lstmNetworkInputSequence(Sequence):
    def __init__(self, train_passwords, password_file_x, password_file_y, batch_size, tokenizer, max_length, vocab_size):
        self.batch_size = batch_size
        self.train_passwords = train_passwords
        self.password_file = password_file_x
        self.password_file_y = password_file_y
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.vocab_size = vocab_size
    
    def __len__(self):
        with open(self.password_file, "r") as pass_file:
            for count, _ in enumerate(pass_file):
                pass
        total_passwords = count + 1
        if(self.train_passwords is not None):
            if(self.train_passwords < total_passwords):
                return math.floor(self.train_passwords / self.batch_size)
            else:
                return math.floor(total_passwords / self.batch_size)
        else:
            return math.floor(total_passwords / self.batch_size)
        
    def __getitem__(self, index):
        # This could be slow, will try to improve speed later by saving encoded passwords.
        # batch_enc_padded_passwords = None
        # batch_y_true = None
        flag = True
        batch_enc_padded_passwords = []
        batch_y_true = []
        # temp_x = []
        # temp_y = []
        batch_passwords = []
        batch_y_passwords = []
        batch_password_index = list(range((index) * self.batch_size, ((index + 1) * self.batch_size)))
        with open(self.password_file, "r") as pass_file:
            for count, password in enumerate(pass_file):
                if(count in batch_password_index):
                    batch_passwords.append(password.rstrip("\n"))
                else:
                    continue
        with open(self.password_file_y, "r") as pass_file_y:
            for count_y, password_y in enumerate(pass_file_y):
                if(count_y in batch_password_index):
                    batch_y_passwords.append(password_y)
                else:
                    continue
        #print(f"{batch_passwords}\n\n\n{batch_y_passwords}")
        encoded_passwords = self.tokenizer.texts_to_sequences(batch_passwords)
        padded_encoded_passwords = pad_sequences(encoded_passwords, padding = "post", maxlen = (self.max_length + 1))
        encoded_y_passwords = self.tokenizer.texts_to_sequences(batch_y_passwords)
        padded_encoded_y_passwords = pad_sequences(encoded_y_passwords, padding = "post", maxlen = (self.max_length + 1))
        for encoded_password, encoded_y_password in zip(padded_encoded_passwords, padded_encoded_y_passwords):
            reshaped_example = np.array(encoded_password).reshape(1, (self.max_length + 1))
            reshaped_y_example = np.array(encoded_y_password).reshape(1, (self.max_length + 1))
            if(flag):
                batch_enc_padded_passwords = reshaped_example
                batch_y_true = to_categorical(y = reshaped_y_example, num_classes = (self.vocab_size + 1))
                flag = False
            else:
                batch_enc_padded_passwords = np.concatenate((batch_enc_padded_passwords, reshaped_example), axis = 0)
                batch_y_true = np.concatenate((batch_y_true, to_categorical(y = reshaped_y_example,  num_classes = (self.vocab_size + 1))), axis = 0)
        return (np.array(batch_enc_padded_passwords).astype(np.float32), np.array(batch_y_true).astype(np.float32))

In [11]:
def createTFDataInputPipeline(X_INPUT_FILE_PATH, Y_INPUT_FILE_PATH, TOKENIZER, MAX_LENGTH, BUFFER_SIZE, PREFETCH_BATCHES, BATCH_SIZE):
    x_inputs = []
    y_inputs = []
    with open(X_INPUT_FILE_PATH, "r") as x_pass_file:
        with open(Y_INPUT_FILE_PATH, "r") as y_pass_file:
            for count, x_y_pass in enumerate(zip(x_pass_file, y_pass_file)):
                x_pass = x_y_pass[0].rstrip("\n")
                y_pass = x_y_pass[1]
                x_inputs.append(x_pass)
                y_inputs.append(y_pass)
    
    # All passwords read in memory:
    x_tokenized = TOKENIZER.texts_to_sequences(x_inputs)
    y_tokenized = TOKENIZER.texts_to_sequences(y_inputs)
    
    x_padded = pad_sequences(x_tokenized, padding = "post", maxlen = (MAX_LENGTH + 1))
    y_padded = pad_sequences(y_tokenized, padding = "post", maxlen = (MAX_LENGTH + 1))
    
    x_np_tokenized = np.array(x_padded)
    y_np_tokenized = np.array(y_padded)
    
    x_dataset = tf.data.Dataset.from_tensor_slices(x_np_tokenized)
    y_dataset = tf.data.Dataset.from_tensor_slices(y_np_tokenized)
    
    fit_dataset = tf.data.Dataset.zip((x_dataset, y_dataset))
    fit_dataset = fit_dataset.shuffle(buffer_size = BUFFER_SIZE)
    fit_dataset = fit_dataset.cache()
    fit_dataset = fit_dataset.batch(BATCH_SIZE)
    fit_dataset = fit_dataset.prefetch(PREFETCH_BATCHES)
    
    return fit_dataset

In [ ]:
# 1. Preparing the Data to splitting into train and validation
# 2. Then prepare the train Data to split again into train and test
# 3. Don't train on test 
# 4. Modify hyperparameters on validate
# 5. Plot using TensorBoard for both train and validation for each epoch 
# 6. Necessary to train if for atleast one epoch, to plot the graphs
# Or we can choose to plot for each batch - but will be very
# Resource intensive and will slow us down considerably.
# 7. train_test_split is given the input and output arrays as X & Y
X_train_test_train, X_validation, Y_train_test_train, Y_validation = train_test_split(inputPasswords, outPasswords, test_size = 0.05, shuffle = True)
X_train, X_test, Y_train, Y_test = train_test_split(X_train_test_train, Y_train_test_train, test_size = 0.1, shuffle = True)

# Sanity Check
print(f"{repr(X_train_test_train[100])}\t{repr(Y_train_test_train[100])}")
print(f"{repr(X_validation[100])}\t{repr(Y_validation[100])}")
print(f"{repr(X_train[100])}\t{repr(Y_train[100])}")
print(f"{repr(X_test[100])}\t{repr(Y_test[100])}")

In [12]:
# Preparing to save models - dedicated methods for it
# We also save them after each epoch using checkpoint callback.
# So saving manually is optional.
def saveKerasModel(MODEL, OUTPUT_MODEL_PATH):
    # Saves the model to the disk, saves both the architecture and 
    # the configuration.
    try:
        assert isinstance(MODEL, Model)
        MODEL.save(OUTPUT_MODEL_PATH)
        print(f"[+] Model has been successfully saved to {OUTPUT_MODEL_PATH}")
    except:
        raise Exception("Model instance is incorrect. Failed!")
        return   

In [13]:
# Utility Function to load the model manually.
# Is Extremely Important!
def loadKerasModel(INPUT_MODEL_PATH):
    loaded_model = tf.keras.models.load_model(INPUT_MODEL_PATH)
    return loaded_model

In [15]:
!mkdir train
!mkdir test
!mkdir validation

A subdirectory or file train already exists.
A subdirectory or file test already exists.
A subdirectory or file validation already exists.


In [14]:
# Utility Function to write the output to file -- 
def writeOutput(FILE_PATH, FILE_TYPE_TRAIN, PASS_LIST_X, PASS_LIST_Y):
    with open(os.path.join(FILE_PATH, FILE_TYPE_TRAIN + "_X" + ".txt"), "w") as x_file:
        for input_password in PASS_LIST_X:
            x_file.write(input_password + "\n")
    with open(os.path.join(FILE_PATH, FILE_TYPE_TRAIN + "_Y" + ".txt"), "w") as y_file:
        for out_password in PASS_LIST_Y:
            y_file.write(out_password)
    print("[+] Done!")

In [ ]:
# Save the split data into their respective directories --
writeOutput(".\\train", "train", X_train, Y_train)
writeOutput(".\\test", "test", X_test, Y_test)
writeOutput(".\\validation", "validation", X_validation, Y_validation)

In [18]:
# Sanity Check 
!head -n 2 ./train/train_X.txt 
!head -n 2 ./train/train_Y.txt
!head -n 2 ./test/test_X.txt 
!head -n 2 ./test/test_Y.txt 
!head -n 2 ./validation/validation_X.txt 
!head -n 2 ./validation/validation_Y.txt 
!wc -l ./train/train_X.txt 
!wc -l ./train/train_Y.txt 
!wc -l ./test/test_X.txt
!wc -l ./test/test_Y.txt
!wc -l ./validation/validation_X.txt 
!wc -l ./validation/validation_Y.txt 

	ellatubar
	tupapiruloybofo
ellatubar
tupapiruloybofo
	c.u2033
	goldsun414
c.u2033
goldsun414
	242518
	1001338
242518
1001338
12060370 ./train/train_X.txt
12060370 ./train/train_Y.txt
1340042 ./test/test_X.txt
1340042 ./test/test_Y.txt
705285 ./validation/validation_X.txt
705285 ./validation/validation_Y.txt


In [17]:
# Preparing the file directory structure for callbacks:
!mkdir checkpoints
!mkdir tensorboard_log_dir
!mkdir tensorboard_checkpoints

A subdirectory or file checkpoints already exists.
A subdirectory or file tensorboard_log_dir already exists.
A subdirectory or file tensorboard_checkpoints already exists.


In [15]:
# Creating Optimizer Instance for Training Model
adam_optimizer = Adam(learning_rate = LEARNING_RATE, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-07)

In [16]:
# Building teacher-forcing training model
# Must include shared architecture for following layers --
# 1. Embedding Layer
# 2. LSTM_1 
# 3. LSTM_2
# 4. LSTM_3
# 5. Dense_1
# 6. Dense_2 (Softmax - Outputs Probability Distribution over |VOCAB_SIZE + <PAD_TOKEN>|)

# The shared layers are defined below -- 
shared_embedding_layer = Embedding(input_dim = (VOCAB_SIZE + 1), output_dim = EMBEDDING_DIMENSION, mask_zero = True, name = "Embedding_Layer")
shared_lstm_1 = LSTM(LSTM_UNITS, return_sequences = True, return_state = True)
shared_lstm_2 = LSTM(LSTM_UNITS, return_sequences = True, return_state = True)
shared_lstm_3 = LSTM(LSTM_UNITS, return_sequences = True, return_state = True)
shared_dense_1 = TimeDistributed(Dense(DENSE_UNITS, activation = "relu"))
shared_dense_op = TimeDistributed(Dense((VOCAB_SIZE + 1), activation = "softmax"))

In [17]:
# 1. Defining the Teacher-Force Training Model:
# 2. Do not use initial_state call argument for the LSTM layers
# 3. Use the initial_state call arguemnt for 'all' the LSTM layers 
# in the Inference Model

train_input = Input(shape = ((MAX_LENGTH + 1),))
train_emb_op = shared_embedding_layer(train_input)
train_lstm_1_op, train_lstm_1_hidden, train_lstm_1_cell = shared_lstm_1(train_emb_op)
train_lstm_2_op, train_lstm_2_hidden, train_lstm_2_cell = shared_lstm_2(train_lstm_1_op)
train_lstm_3_op, train_lstm_3_hidden, train_lstm_3_cell = shared_lstm_3(train_lstm_2_op)
train_dense_1_op = shared_dense_1(train_lstm_3_op)
train_model_op = shared_dense_op(train_dense_1_op)

train_model = Model(inputs = train_input, outputs = train_model_op)

# Check the model summary 
print(train_model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 33)]              0         
_________________________________________________________________
Embedding_Layer (Embedding)  (None, 33, 16)            1136      
_________________________________________________________________
lstm (LSTM)                  [(None, 33, 300), (None,  380400    
_________________________________________________________________
lstm_1 (LSTM)                [(None, 33, 300), (None,  721200    
_________________________________________________________________
lstm_2 (LSTM)                [(None, 33, 300), (None,  721200    
_________________________________________________________________
time_distributed (TimeDistri (None, 33, 100)           30100     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 33, 71)            7171  

In [18]:
# Compiling the Model with specified callbacks and metrics:
train_model.compile(optimizer = adam_optimizer, loss = tf.keras.losses.SparseCategoricalCrossentropy(), metrics = METRICS)

In [19]:
# Defining the Inference Model
# Needs the initial_state call argument for all the LSTM layers

inference_input = Input(shape = (1,))
lstm_1_hidden = Input(shape = (LSTM_UNITS,))
lstm_1_cell = Input(shape = (LSTM_UNITS,))
lstm_2_hidden = Input(shape = (LSTM_UNITS,))
lstm_2_cell = Input(shape = (LSTM_UNITS,))
lstm_3_hidden = Input(shape = (LSTM_UNITS,))
lstm_3_cell = Input(shape = (LSTM_UNITS,))
inference_emb_op = shared_embedding_layer(inference_input)
inference_lstm_1_op, inference_lstm_1_hidden, inference_lstm_1_cell = shared_lstm_1(inference_emb_op, initial_state = [lstm_1_hidden, lstm_1_cell])
inference_lstm_2_op, inference_lstm_2_hidden, inference_lstm_2_cell = shared_lstm_2(inference_lstm_1_op, initial_state = [lstm_2_hidden, lstm_2_cell])
inference_lstm_3_op, inference_lstm_3_hidden, inference_lstm_3_cell = shared_lstm_3(inference_lstm_2_op, initial_state = [lstm_3_hidden, lstm_3_cell])
inference_dense_1_op = shared_dense_1(inference_lstm_3_op)
inference_model_op = shared_dense_op(inference_dense_1_op)

inputs_list = [inference_input, lstm_1_hidden, lstm_1_cell, lstm_2_hidden, lstm_2_cell, lstm_3_hidden, lstm_3_cell]
outputs_list = [inference_model_op, inference_lstm_1_hidden, inference_lstm_1_cell, inference_lstm_2_hidden, inference_lstm_2_cell, inference_lstm_3_hidden, inference_lstm_3_cell]

inference_model = Model(inputs = inputs_list, outputs = outputs_list)

print(inference_model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
Embedding_Layer (Embedding)     multiple             1136        input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 300)]        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 300)]        0                                            
____________________________________________________________________________________________

In [20]:
inference_model.compile(optimizer = "adam", loss = tf.keras.losses.SparseCategoricalCrossentropy(), metrics = METRICS)

In [21]:
# Creating a metadata file for tensorboard embedding visualization
def createEmbeddingMetadata(TOKENIZER):
    tokenized_chars = [token for token, idx in TOKENIZER.word_index.items()]
    with open(EMBEDDING_METADATA, 'w') as emb_file:
        emb_file.write("<PAD>\n")
        for token in tokenized_chars:
            if (token == '\n'):
                emb_file.write("\\n\n")
                continue
            if(token == '\t'):
                emb_file.write("\\t\n")
                continue
            emb_file.write(token+"\n")

createEmbeddingMetadata(passwordTokenizer)

In [22]:
# Configuring all model callbacks
# 1. LearningRate Callback
# 2. ModelCheckpoint Callback
# 3. TensorBoard Callback
# 4. GradientClipping Callback
# Gradient Clipping is required because LSTMs/GRUs greatly suffer from both
# Exploding and Vanishing Gradients which presents as big problem and sets
# all values of delta-wt, delta-b to NaN causing numerical instability.
# This occurs because of Gradient-Descent through Time or as we call it
# Back-Propogation through time algorithm causing the gradients to be multiplied 
# with <1 or >1 weight values across all timesteps leaving the network extremely
# vulnerable to numerical instability.

# LearningRateScheduler - Using Exponential Learning Rate Decay
# First defining a 'schedule' for learning rate decay
def expLearningRateDecay(epoch):
   initial_lrate = LEARNING_RATE
   k = 0.1
   lrate = initial_lrate * math.exp(-k * epoch)
   return lrate

# Scheduler Callback:
train_learning_rate_callback = LearningRateScheduler(schedule = expLearningRateDecay, verbose = 1)

# Checkpoint Callback:
train_checkpoint_callback = ModelCheckpoint(".\\checkpoints\\Checkpoint-{epoch:03d}", verbose = 1, save_weights_only = False, save_freq = "epoch")

# TensorBoard Callback:
train_tensorboard_callback = TensorBoard(log_dir = ".\\tensorboard_log_dir", histogram_freq = 1, write_graph = True, write_images = True, update_freq = 'epoch', embeddings_freq = 1)



# Custom Callback
# Tf stable version 2.x currently has a bug
# with saved_model format of saving models.
# Loading of saved_model formatted serialized
# model requires tf-nightly-gpu version which 
# contains latest bug fixes.
# We require a custom callback that saves the 
# entire model in h5 format which can be loaded
# without any problem.

# We will also implement to save inference_model 
# at another directory everytime. 

# This custom callback will work with the newer
# saved_model format, thus allowing flexibility.

class saveTensorboardCheckpoint(tf.keras.callbacks.Callback):
    def __init__(self):
        self.save_tensorboard_checkpoint = False
        self.epoch = None
        
    def on_epoch_begin(self, epochs, logs = None):
        self.epoch = epochs
        if(not self.save_tensorboard_checkpoint):
            self.save_tensorboard_checkpoint = True
            return
        else:
            path = ".\\tensorboard_checkpoints\\Tensorboard-Checkpoint-{epoch:03d}".format(epoch = epochs)
            if(os.path.isdir(path)):
                return
            else:
                shutil.copytree(".\\tensorboard_log_dir", path)
        
    def on_train_begin(self, logs=None):
        self.save_tensorboard_checkpoint = False
    
    def on_train_end(self, logs=None):
        path = ".\\tensorboard_checkpoints\\Tensorboard-Checkpoint-{epoch:03d}".format(epoch = self.epoch + 1)
        if(os.path.isdir(path)):
            return
        else:
            shutil.copytree(".\\tensorboard_log_dir", path)

class saveH5CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epochs, logs = None):
        # Saving the train model in h5 format. This should always be defined.
        self.model.save(filepath = ".\\h5_train_checkpoints\\H5-Checkpoint-{epoch:03d}.h5".format(epoch = (epochs + 1)), include_optimizer = True, save_format = "h5")
        try:
            # Check if the inference_model is defined.
            # If defined save the model in h5 format
            global inference_model
            inference_model.save(filepath = ".\\h5_inference_checkpoints\\H5-Inference-Checkpoint-{epoch:03d}.h5".format(epoch = (epochs + 1)), include_optimizer = True, save_format = "h5")
        except:
            # If Inference Model not defined, log the message.
            with open("inference_log.txt", "a") as inf_err:
                inf_err.write("Inference Model not defined for epoch {epoch:03d}.\nSaving train model only. Build Inference Model Later.\n\n".format(epoch = (epochs + 1)))

        
# Creating an instance of custom callback.    
saveH5Callback = saveH5CustomCallback()
saveTensorboardCheckpointCallback = saveTensorboardCheckpoint()

# Compiling all callback together..
callbacks = [saveTensorboardCheckpointCallback, train_checkpoint_callback, train_tensorboard_callback, saveH5Callback]

# GradientClipping Callback

# GradientClipping has to be done manually by computing the gradient wrt 
# the loss of each trainable parameter and then clip it.
# After which the gradients have to be applied to the weights / biases / gates
# thus preventing exploding/vanishing gradient.

# This will only be done if the model experiences any sort of numerical 
# instability and throws NaN Exceptions since this process has capacity 
# to cripple model's speed.


In [23]:
# Load the validation data into volatile memory so we 
# don't have to re-perform all the calculation during each 
# validation test run.
# Fitting the model might be very resource intensive
# Due to data fetch from physical disk and prior pre-processing.


###################################################################

# WARNING:
# Sequence class objects as input to the models suffer from
# extensive deadlocks. Coupling this with model.fit arg 
# use_multiprocessing = True can failed memory allocation
# process during fitting. 
# For flexibility, the sequence objects have been implemented.
# However, their use is not recommended as they are unstable 
# and slow down the pre-fetching capabilities.
# tf.data.Dataset implementations are preferred and are defined 
# above. Please use those to implement input and output pipelines
# for both the training and validation sets. 

####################################################################

# DO NOT EXECUTE.
input_pipeline = lstmNetworkInputSequence(10000, ".\\train\\train_X.txt", ".\\train\\train_Y.txt", BATCH_SIZE, passwordTokenizer, MAX_LENGTH, VOCAB_SIZE)
print("[+] Sequence Object Created!")
validation_data = getValidationData(".\\validation\\validation_X.txt", ".\\validation\\validation_Y.txt", VOCAB_SIZE, passwordTokenizer, MAX_LENGTH, BATCH_SIZE, 2000)
print("[+] Validation Data is read into volatile memory!")

[+] Sequence Object Created!
1984
1984
Total passwords processed 500
Total passwords processed 1000
Total passwords processed 1500
[+] Validation Data is read into volatile memory!


In [23]:
train_ip_pipeline = createTFDataInputPipeline(".\\train\\train_X.txt", ".\\train\\train_Y.txt", passwordTokenizer, MAX_LENGTH, SHUFFLE_BUFFER_SIZE, PREFETCH_BATCHES, BATCH_SIZE)
validation_ip_pipeline = createTFDataInputPipeline(".\\validation\\validation_X.txt", ".\\validation\\validation_Y.txt", passwordTokenizer, MAX_LENGTH, SHUFFLE_BUFFER_SIZE, PREFETCH_BATCHES, BATCH_SIZE)

In [34]:
train_model_history = train_model.fit(x = train_ip_pipeline, epochs = EPOCHS, callbacks = callbacks, verbose = VERBOSITY, validation_data = validation_ip_pipeline, validation_freq = 1)
# Callbacks and validation will be done every epoch.
# To change this, 
# change definitions for callbacks &,
# change the validation_freq 
# arg in model.fit

Train for 376887 steps, validate for 22041 steps
Epoch 1/50
376884/376887 [============================>.] - ETA: 0s - loss: 0.7391 - sparse_categorical_crossentropy: 2.5717 - accuracy: 0.2597
Epoch 00001: saving model to .\checkpoints\Checkpoint-001
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: .\checkpoints\Checkpoint-001\assets
376887/376887 [==============================] - 5280s 14ms/step - loss: 0.7391 - sparse_categorical_crossentropy: 2.5717 - accuracy: 0.2597 - val_loss: 0.7179 - val_sparse_categorical_crossentropy: 2.4981 - val_accuracy: 0.2799
Epoch 2/50
376884/376887 [============================>.] - ETA: 0s - loss: 0.7132 - sparse_categorical_crossentropy: 2.4816 - accuracy: 0.2840
Epoch 00002: saving model to .\checkpoints\Checkpoint-002
INFO:tensorflow:Assets written to: .\checkpoints\Checkpoint-002\assets
376887/376887 [==============================] - 5418s 14ms/step - loss: 0.7132 - sparse_catego

KeyboardInterrupt: 

In [24]:
def restart_learning_with_lr(epochs_completed, revised_lr, using_checkpoint = False, checkpoint_path = False):
    train_model = tf.keras.models.load_model(".\\h5_train_checkpoints\\H5-Checkpoint-{_epochs_completed:03d}.h5".format(_epochs_completed = epochs_completed))
    inference_model = tf.keras.models.load_model(".\\h5_inference_checkpoints\\H5-Inference-Checkpoint-{_epochs_completed:03d}.h5".format(_epochs_completed = epochs_completed))
    global callbacks
    global train_ip_pipeline
    global EPOCHS
    global validation_ip_pipeline
    global VERBOSITY
    K.set_value(train_model.optimizer.learning_rate, revised_lr)
    train_model_history = train_model.fit(x = train_ip_pipeline, initial_epoch = epochs_completed, epochs = EPOCHS, callbacks = callbacks, verbose = VERBOSITY, validation_data = validation_ip_pipeline, validation_freq = 1)
    return train_model_history

In [25]:
# Deletes every subdir inside of the supplied directory
def delete_subdirs(supplied_dir):
    folder = supplied_dir
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
# Cleans up everything - util function
def cleanup():
    folders = [".\\checkpoints", ".\\h5_train_checkpoints", ".\\tensorboard_checkpoints", ".\\h5_inference_checkpoints", ".\\tensorboard_log_dir"]
    for folder in folders:
        delete_subdirs(folder)
cleanup()

In [26]:
# Manual setting of learning rate:
#
#
train_model = tf.keras.models.load_model(".\\h5_train_checkpoints\\H5-Checkpoint-{_epochs_completed:03d}.h5".format(_epochs_completed = 40))
inference_model = tf.keras.models.load_model(".\\h5_inference_checkpoints\\H5-Inference-Checkpoint-{_epochs_completed:03d}.h5".format(_epochs_completed = 40))
K.set_value(train_model.optimizer.learning_rate, 0.00006666)
#
# Sanity Check
#
print(train_model.optimizer.learning_rate)


<tf.Variable 'training/Adam/learning_rate:0' shape=() dtype=float32, numpy=6.666e-05>


In [27]:
train_model_history = train_model.fit(x = train_ip_pipeline, initial_epoch = 40, epochs = EPOCHS, callbacks = callbacks, verbose = VERBOSITY, validation_data = validation_ip_pipeline, validation_freq = 1)

Train for 376887 steps, validate for 22041 steps
Epoch 41/50
     1/376887 [..............................] - ETA: 407:32:07 - loss: 0.7195 - sparse_categorical_crossentropy: 2.4590 - accuracy: 0.2945WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.157594). Check your callbacks.


289005/376887 [======================>.......] - ETA: 1:46:06 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4113 - accuracy: 0.3026- ETA: 1:55:22 - ETA: 1:55:04 - loss: 0.6930 - - ETA: 1:55:00 - loss: 0.6930 - sparse_categorical_cross - ETA: 1:54:58 - loss: 0.6930 - - ETA: 1:54:48 - loss: 0 - ETA: 1:54:43 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4114 - accura - ETA: 1:54:43 - ETA: 1:54:37 - loss: 0.6930 - sparse_categorical_cro - ETA: 1:54:35 - loss: 0.6930 - sparse_categorical_crossent - ETA: 1:54:33 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4113 - ac - ETA: 1:54:32 - los - ETA: 1:54:27 - lo - ETA: 1:53:08 - loss: - ETA: 1:53:03 - loss: 0.6 - ETA: 1:52:59 - loss: 0.6930 - sparse_categorical_crossentrop - ETA: 1:52:57 - loss: 0.6930 - sparse_categorical_crossentropy: 2.411 - ETA: 1:52:55 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4114 - accuracy: 0.30 - ETA: 1:52:55 - loss: 0.6930 - sparse_categorical_ - ETA: 1:52:53 - loss: 0.6930 - sparse_categor -

292467/376887 [======================>.......] - ETA: 1:42:08 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4112 - accuracy: 0.3026- ETA: 1:46:04 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4113 - accurac - ETA: 1:46:03 - loss: 0.6930 - sparse_categorical_ - ETA: 1:45:54 - loss: 0.6930 - sparse_catego - ETA: 1:45:51 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4113 - accurac - ETA: 1:45:51 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4113 - accuracy: 0.30 - ETA: 1:45:50 - loss: 0.6930 - sparse_categorical_crosse - ETA: 1:45:48 - loss: 0. - ETA: 1:45:44 - loss: 0.6930 - sparse_categorical_crossentropy: 2.41 - ETA: 1:45:42 - loss: 0.6930 - sparse_c - ETA: 1:45:33 - loss: 0.6930 - sparse_categor - ETA: 1:45:30 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4113 - ETA: 1:45:28 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4113 - accuracy:  - ETA: 1:45:28 - loss: 0.6930 - sparse_categorical_crosse - ETA: 1:45:26 - loss: 0.6930 - sparse_categorical_cros

295117/376887 [======================>.......] - ETA: 1:39:05 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4112 - accuracy: 0.3026- ETA: 1:42:03 - loss: 0.6930 - sparse_ca - ETA: 1:41:53 - loss: 0.6930 - sparse_categorical_cro - ETA: 1:41:51 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4112 - accura - ETA: 1:41:50 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4112 -  - ETA: 1:41:49 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4112 - accu - ETA: 1:41:48 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4112 -  - ETA: 1:41:47 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4112 - accur - ETA: 1:41:46 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4112 - accuracy - ETA: 1:41:46 - loss: 0.6930 - sparse_categorical_crossentro - ETA: 1:41:44 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4112 - accuracy: 0.302 - ETA: 1:41:44 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4112 - accura - ETA: 1:41:43 - loss: 0.6930 - sparse_categorical_crossentr

297699/376887 [======================>.......] - ETA: 1:36:06 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4112 - accuracy: 0.3026- ETA: 1:39:03 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4112 - accuracy: 0 - ETA: 1:39:03 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4112 - accuracy: 0.3 - ETA: 1:39:03 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4112 - accuracy: - ETA: 1:39:02 - loss: 0.6930 - sparse_categorical_crossentr - ETA: 1:39:00 - loss: 0.6930 - sparse_ca - ETA: 1:38:57 - loss: 0.6930 - sparse_categorical_c - ETA: 1:38:54 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4112 - accuracy: 0.30 - ETA: 1:38:54 - loss: 0.6930 - sparse_categorical_crossentrop - ETA: 1:38:52 - loss - ETA: 1:38:47 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4112 - accuracy: 0.30 - ETA: 1:38:47 - loss: 0.6930 - sparse_categorical_crossentropy: 2.4112 - accuracy:  - ETA: 1:38:47 - loss: 0.6930 -  - ETA: 1:38:43 - loss: 0.6930 - s - ETA: 1:38:39 - loss: 0.6930 - sp

...

365772/376887 [============================>.] - ETA: 20:00 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: 0.3042- ETA: 26:03 - loss: 0.6912 -  - ETA: 25:10 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4 - ETA - ETA: 24:59 - - ETA: 24:51 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accur - ETA: 24:50 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - acc - ETA: 24:49 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy - ETA: 24:48 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: 0 - ETA: 24:47 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: 0.304 - ETA: 24:47 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - a - ETA: 24:46 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - - ETA: 24:44 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accu - ETA: 24:24 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - - ETA: 24:22 - loss: 0.6912 - sparse_ca - ET

367850/376887 [============================>.] - ETA: 16:19 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: 0.3042- ETA: 19:59 - loss: 0.6912 - sparse_catego - ETA: 19:55 - loss: 0.6912 - sparse_categorical_crossentropy - ETA: 19:52 - loss: 0.6912 - sparse_categorical_crossentropy - ETA: 19:49 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accur - ETA: 19:48 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: 0.304 - ETA: 19:48 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: 0.304 - ETA: 19:48 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy - ETA: 19:47 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy:  - ETA: 19:46 - loss: 0.6912 - sparse_categorical_cro - ETA: 19:42 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: 0 - ETA: 19:42 - loss: 0. - ETA: 19:35 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: 0. - ETA: 19:35 - loss: 0.6912 - spa

370552/376887 [============================>.] - ETA: 11:30 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: 0.3042- ETA: 16:05 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: 0.30 - ETA: 16:05 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: 0.304 - ETA: 16:05 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: 0.304 - ETA: 16:05 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: - ETA: 16:04 - loss: 0.6912 - sparse_categorical_crossentro - ETA: 16:01 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: 0.30 - ETA: 16:00 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - ETA: 15:58 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: 0.30 - ETA: 15:58 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: - ETA: 15:57 - loss: 0.6912 - sparse_categorical_ - ETA: 15:53 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy:  -

374359/376887 [============================>.] - ETA: 4:37 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: 0.3042 - ETA: 11:29 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accura - ETA: 11:28 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accura - ETA: 11:27 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accura - ETA: 11:26 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: 0.3 - ETA: 11:25 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4 - ETA: 11:23 - loss: 0.6912 - spar - ETA: 11:08 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accura - ETA: 11:07 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accu - ETA: 11:05 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: 0.30 - ETA: 11:05 - lo - ETA: 10:57 - loss: 0.6912 - sparse_categor - ETA: 10:53 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: - ETA: 10:52 - loss: 0.6912 - sparse_categorical_cro

376886/376887 [============================>.] - ETA: 0s - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: 0.3042   ETA: 4:32 - loss: 0.6912 - sparse_categorical_crossen - ETA: 4:30 - loss: 0.6912 - sparse_categorical_crossentropy: - ETA: 4:29 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: 0. - ETA: 4:28 - loss: 0.6912 - sparse_categorical - ETA: 4:26 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accura - ETA: 4:26 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy:  - ETA: 4:25 - loss: 0.6912 - sparse_categorical_crossentropy: 2 - ETA: 4:24 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy - ETA: 4:14 - loss: 0.6912 - sparse_categ - ETA: 4:11 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy:  - ETA: 4:11 - loss: 0 - ETA: 4:08 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 -  - ETA: 4:07 - loss: 0.6912 - sparse_categorical_crossentropy: 2.4049 - accuracy: 0.30 - ETA: 4: